In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.part'):
    shutil.rmtree('./file/3.part')

os.makedirs('./file/3.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
                            if type_:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_src, i, list_src = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
                for src in list_src:
                    if src.startswith('http'):
                        dict_src[str(i)] = src
                        i += 1

                # = = = = = = = = = = = = = = =

                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
                while '' in list_text:
                    list_text.remove('')
                
                application_summary = ';'.join(list_text)

                # = = = = = = = = = = = = = = =

                brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =

                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                vehicle_url = f'''https://www.dormanproducts.com/{vehicle_url[0].strip()}''' if vehicle_url else ''

                # = = = = = = = = = = = = = = =

                list_tr, dict_count = html.xpath('//section[@id="productSpec"]/div/table/tr'), {}
                for tr in list_tr:
                    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                        dict_count[key] = 1
                
                dict_specification, dict_no = {}, {}
                for i, tr in enumerate(list_tr):
                    key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
                    if dict_count[key] == 1:
                        dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
                    else:
                        if key in dict_no:
                            dict_no[key] += 1
                        else:
                            dict_no[key] = 1
                        dict_specification[str(i)] = {f'''{key}-{dict_no[key]}''': tr.xpath('./td/text()')[0].strip()}

                # = = = = = = = = = = = = = = =

                list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
                oem = ';'.join(sorted(list(set(list_text)), key=list_text.index))

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': type_,
                                         'Part Number': part_number,
                                         'Brand': brand,
                                         'Description': description,
                                         'Application Summary': application_summary,
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': request_url,
                                         'Json_Src': json.dumps(dict_src),
                                         'Vehicle Url': vehicle_url,
                                         'Json_Specification': json.dumps(dict_specification)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Type', 'Part Number'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct['No'] = [i+1 for i in range(len(output_correct))]

        output_correct.to_excel(f'''./file/3.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.part/{file}',
              f'''./file/3.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.dormanproducts.com/p-21537-w72258.aspx?origin=keyword
[剩余数量：5] - [当前时间：13:53:28]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.dormanproducts.com/p-50659-w610165.aspx?origin=keyword
[剩余数量：4] - [当前时间：13:53:28]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://www.dormanproducts.com/p-47833-w610150.aspx?origin=keyword
[剩余数量：3] - [当前时间：13:53:28]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.dormanproducts.com/p-9421-w37425.aspx?origin=keyword
[剩余数量：2] - [当前时间：13:53:28]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.dormanproducts.com/p-50424-w610166.aspx?origin=keyword
[剩余数量：1] - [当前时间：13:53:29]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 8.https://www.dormanproducts.com/p-15216-w37872.aspx?origin=keyword
[剩余数量：0] - [当前时间：13:53:30]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.https://www.dormanproducts.com/p-9423-w37635.aspx?origin=keyword
[剩余数量：0] - [当前时间：13:53:30]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://www.d

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3092.00it/s]

Done ~
